In [124]:
import collections
import csv
import os
import pyreadr
import sklearn
import scipy


import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing, metrics

LEARNING_RATE = 0.005
EPOCHS = 1000
MOMENTUM = 0.75
NEURONS = 1000
BATCHES = 256
VALIDATION = 0.1
TARGET_NAME = "log2relT"
DATA_DIR = "../../../data/models/moma/"

In [125]:
def init_single_view_model(
    input_dim: int,
    model_name: str,
) -> tf.keras.Model:
    """Initialize a model with the given parameters.

    Parameters
    ----------
    input_dim : int
        The number of input features.
    model_name : str
        The name of the model. Used for naming the layers.

    Returns
    -------
    tf.keras.Model
    """
    # Input layer
    input = tf.keras.layers.Input(shape=(input_dim,))

    # Hidden layer (1)
    layer = tf.keras.layers.Dense(
        NEURONS,
        activation="sigmoid",
        kernel_constraint=tf.keras.constraints.max_norm(3),
        name=f"{model_name}_1",
    )(input)
    # Set 40% of input units to 0 at each update during training time
    layer = tf.keras.layers.Dropout(rate=0.4)(layer)

    # Hidden layer (2)
    layer = tf.keras.layers.Dense(
        NEURONS,
        activation="sigmoid",
        kernel_constraint=tf.keras.constraints.max_norm(3),
        name=f"{model_name}_2",
    )(layer)
    # Set 40% of input units to 0 at each update during training time
    layer = tf.keras.layers.Dropout(rate=0.4)(layer)

    # Final output layer
    predictions = tf.keras.layers.Dense(1, activation="linear")(layer)
    model = tf.keras.Model(inputs=input, outputs=predictions)
    print(f"Summary of the single-view model {model_name}")
    model.summary()
    return model

In [126]:
def load_single_view_model_from_file(
    training_data: np.ndarray, fname: str
) -> tf.keras.Model:
    """Load a model from a file and return it.

    Parameters
    ----------
    data_train : np.ndarray
        The training data to load the model for.
    fname : str
        The name of the model to load.

    Returns
    -------
    tf.keras.Model
        The loaded model.

    """
    # Model initialization
    model = init_single_view_model(
        input_dim=training_data.shape[1],
        model_name=fname,
    )

    # Load the model weights
    #   - gene expression: gene_expression_weights.h5
    #   - fluxomic: fluxomic_weights.h5
    model.load_weights(DATA_DIR + fname + "_weights.h5")

    return model

In [127]:
def init_triple_view_model(
    input1_dim: int,
    input2_dim: int,
    input3_dim: int,
    learning_rate: float,
    epochs: int,
    momentum: float,
    neurons: int,
    model_1: tf.keras.Model,
    model_2: tf.keras.Model,
    model_3: tf.keras.Model,
) -> tf.keras.Model:
    """Initialize a model with two inputs and one output.

    Parameters
    ----------
    input1_dim : int
        The number of features in the first input.
    input2_dim : int
        The number of features in the second input.
    input3_dim : int
        The number of features in the third input.
    learning_rate : float
        The learning rate for the optimizer.
    epochs : int
        The number of epochs for training.
    momentum : float
        The momentum for the optimizer.
    neurons : int
        The number of neurons in the hidden layers.
    model_1 : tf.keras.Model
        The first model to use.
    model_2 : tf.keras.Model
        The second model to use.
    model_3 : tf.keras.Model
        The third model to use.

    Returns
    -------
    tf.keras.Model

    """
    input_1 = tf.keras.layers.Input(shape=(input1_dim,))
    input_2 = tf.keras.layers.Input(shape=(input2_dim,))
    input_3 = tf.keras.layers.Input(shape=(input3_dim,))

    combined_layer = tf.keras.layers.Concatenate()(
        [model_1(input_1), model_2(input_2), model_3(input_3)]
    )
    combined_layer = tf.keras.layers.Dense(
        neurons,
        activation="sigmoid",
        kernel_constraint=tf.keras.constraints.max_norm(3),
        name="last_hidden",
    )(combined_layer)

    predictions = tf.keras.layers.Dense(1, activation="linear")(combined_layer)
    result = tf.keras.Model(inputs=[input_1, input_2, input_3], outputs=predictions)

    sgd = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        weight_decay=learning_rate / epochs,
        momentum=momentum,
    )
    result.compile(
        loss="mean_squared_error", optimizer=sgd, metrics=["mean_absolute_error"]
    )
    return result

In [128]:
def compute_metrics(predicted, true) -> dict:
    if predicted.ndim > 1:
        predicted = predicted.ravel()  # Flatten the array
    # Check for NaN and handle them
    true = np.nan_to_num(true)
    predicted = np.nan_to_num(predicted)

    mse = float(metrics.mean_squared_error(y_true=true, y_pred=predicted))
    residual = true - predicted
    residual_sum_of_squares = np.sum(np.square(residual))
    total_sum_of_squares = np.sum(np.square(true - np.mean(true)))
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)
    pearson, _ = scipy.stats.pearsonr(true, predicted)
    spearman, _ = scipy.stats.spearmanr(true, predicted)

    non_nan_count = np.count_nonzero(~np.isnan(predicted))
    total_count = len(predicted)
    coverage = float(non_nan_count / total_count)
    return {
        "mse": mse,
        "pearson": pearson,
        "spearman": spearman,
        "coverage": coverage,
        "r_squared": r_squared,
    }

In [129]:
# Function to rename columns based on your specifications
def rename_column(col) -> str:
    if "_ko_" in col:
        # Split the string at "_ko_" and take the part after it
        parts = col.split("_ko_")
        # Further split by "_" and take the first element which is the systematic gene name
        gene_name = parts[1].split("_")[0]
        return gene_name
    return col  # Return the original name if "_ko_" is not in the column name

In [130]:
def check_file_read_ok(data: collections.OrderedDict, name: str) -> None:
    if data is None:
        print("error with " + name + " data not read")
    else:
        print("Success in loading " + name)
        print(data.shape)

In [131]:
# IMPORTING - Load names mapping
names_systematic_to_standard = pd.read_csv("yeast_gene_names.tsv", sep="\t")
# Create a dictionary where both standard and systematic names map to systematic names
mapping_dict = pd.Series(
    names_systematic_to_standard["systematic_name"].values,
    index=names_systematic_to_standard["standard_name"],
).to_dict()
# Also add systematic names as keys mapping to themselves
mapping_dict.update(
    pd.Series(
        names_systematic_to_standard["systematic_name"].values,
        index=names_systematic_to_standard["systematic_name"],
    ).to_dict()
)

# IMPORTING - GE_MF DATASET
data_GE_MF_raw: collections.OrderedDict = pyreadr.read_r(DATA_DIR + "complete_dataset.RDS")[
    None
]
check_file_read_ok(data=data_GE_MF_raw, name="data_GE_MF_raw")
# Remove columns consisting of only zeros
data_GE_MF = data_GE_MF_raw.loc[:, (data_GE_MF_raw != 0).any(axis=0)]
# Convert to sytematic names and get names and growth rates
data_GE_MF["Row"] = data_GE_MF["Row"].map(mapping_dict).fillna(data_GE_MF["Row"])

growth_data = data_GE_MF[["Row", "log2relT"]]  # 1143 rows, 2 columns

# IMPORTING - testing indices
with open(DATA_DIR + "indices_for_testing_data.csv", "r") as csvfile:
    testing_data_indices = []
    for row in csv.reader(csvfile, delimiter=";"):
        testing_data_indices.append(row[0])  # careful here with [0]
testing_data_indices = list(map(int, testing_data_indices))  # fixed testing indexes

# IMPORTING - GE dataset
transcriptomics_data_raw = pyreadr.read_r(DATA_DIR + "gene_expression_dataset.RDS")[None]
check_file_read_ok(transcriptomics_data_raw, "expression data")
# Remove columns consisting of only zeros
transcriptomics_data = transcriptomics_data_raw.loc[
    :, (transcriptomics_data_raw != 0).any(axis=0)
]
transcriptomics_data["knockout_id"] = growth_data["Row"]

proteomics_data = pd.read_csv("../../../data/models/moma/yeast5k_impute_wide.csv")
# Row = 1850 (protein), Column = 5477 (gene knockout)

Success in loading data_GE_MF_raw
(1143, 9666)


/var/folders/ch/ypsvlmcx5zl0s9mxmw1v795c0000gn/T/ipykernel_72440/1791084953.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_GE_MF["Row"] = data_GE_MF["Row"].map(mapping_dict).fillna(data_GE_MF["Row"])


Success in loading expression data
(1143, 6171)


In [132]:
# PREPROCESSING - REMOVE CONTROLS

# Remove the quality control columns (they contain "_qc_qc_qc")
quality_control_data = proteomics_data.filter(regex="_qc_qc_qc", axis=1)  # 389 columns
proteomics_data = proteomics_data.drop(
    quality_control_data.columns, axis=1
)  # 5088 columns

# Remove the his3Δ control strains (388 replicates)
his3_control_data = proteomics_data.filter(regex="_HIS3", axis=1)  # 388 columns
proteomics_data = proteomics_data.drop(
    his3_control_data.columns, axis=1
)  # 4700 columns


# PREPROCESSING - RENAME COLUMNS

# Apply the renaming function to all columns except the first two
new_columns = [
    proteomics_data.columns[i] if i < 1 else rename_column(proteomics_data.columns[i])
    for i in range(len(proteomics_data.columns))
]
proteomics_data.columns = new_columns

# Set the first column as the index
proteomics_data.set_index(proteomics_data.columns[0], inplace=True)  # 4699 columns
# change the index name to "protein_id"
proteomics_data.index.name = "protein_id"

# # PREPROCESSING - ROTATE DATA

# # Transpose the data so that the rows are knockouts and the columns are proteins
proteomics_data = proteomics_data.T  # 4699 rows, 1850 columns

In [133]:
# INTERSECTION STEP - intersect two dataframes
common_knockouts = proteomics_data.index.intersection(growth_data["Row"])
print(f"Number of common knockouts: {len(common_knockouts)}")

aligned_proteomics_data = proteomics_data.loc[
    common_knockouts
]  # 1069 rows (there are duplicates)
aligned_growth_data = growth_data[growth_data["Row"].isin(common_knockouts)].set_index(
    "Row", drop=True
)  # 1038 rows

Number of common knockouts: 1038


In [134]:
# PREPROCESSING - remove duplicates
# check if the index in aligned_proteomics_data is duplicated
print(
    f"Number of duplicated indexes in aligned_proteomics_data: {aligned_proteomics_data.index.duplicated().sum()}"
)
# keep only one of the duplicates
aligned_proteomics_data = aligned_proteomics_data[
    ~aligned_proteomics_data.index.duplicated(keep="first")
]  # 1038 rows

Number of duplicated indexes in aligned_proteomics_data: 31


In [135]:
# PREPROCESSING - Splitting the GE data
# Gene expression data is missing systematic names, add them
transcriptomics_data["knockout_id"] = growth_data["Row"]

# Split the gene expression data into training and testing
transcriptomics_training_data, transcriptomics_testing_data = (
    transcriptomics_data.drop(transcriptomics_data.index[testing_data_indices]),
    transcriptomics_data.iloc[testing_data_indices, :],
)

In [136]:
# PREPROCESSING - Splitting the MF data
transcriptomics_data_no_knockouts = transcriptomics_data.drop(columns=["knockout_id"])
fluxomics_data = data_GE_MF.drop(
    columns=transcriptomics_data_no_knockouts.columns.values
)
fluxomics_data = fluxomics_data.rename(columns={"Row": "knockout_id"})

fluxomic_training_data, fluxomic_testing_data = (
    fluxomics_data.drop(fluxomics_data.index[testing_data_indices]),
    fluxomics_data.iloc[testing_data_indices, :],
)

In [137]:
# PREPROCESSING - Splitting the P data
proteomics_testing_data = aligned_proteomics_data[
    aligned_proteomics_data.index.isin(transcriptomics_testing_data["knockout_id"])
]  # 835
proteomics_training_data = aligned_proteomics_data[
    aligned_proteomics_data.index.isin(transcriptomics_training_data["knockout_id"])
]  # 203

In [138]:
# intersect fluxomic and transcriptomics data with proteomics
fluxomic_training_data = fluxomic_training_data[
    fluxomic_training_data["knockout_id"].isin(proteomics_training_data.index)
]
fluxomic_testing_data = fluxomic_testing_data[
    fluxomic_testing_data["knockout_id"].isin(proteomics_testing_data.index)
]

transcriptomics_training_data = transcriptomics_training_data[
    transcriptomics_training_data["knockout_id"].isin(proteomics_training_data.index)
]
transcriptomics_testing_data = transcriptomics_testing_data[
    transcriptomics_testing_data["knockout_id"].isin(proteomics_testing_data.index)
]

In [139]:
# PREPROCESSING - Splitting the growth data
target_training_data, target_testing_data = (
    growth_data.drop(testing_data_indices),
    growth_data.loc[testing_data_indices],
)
# intersect with proteomics_testing_data by the Row in target_testing_data and protein_id in proteomics_testing_data
target_testing_data = target_testing_data[
    target_testing_data["Row"].isin(proteomics_testing_data.index)
]
# intersect with proteomics_training_data by the Row in target_training_data and protein_id in proteomics_training_data
target_training_data = target_training_data[
    target_training_data["Row"].isin(proteomics_training_data.index)
]

# Convert the target data to float32. For this, remove the Row column
target_training_data = target_training_data.drop(columns=["Row"])
target_testing_data = target_testing_data.drop(columns=["Row"])
target_training_data: pd.Series = target_training_data.astype(np.float32)
target_testing_data: pd.Series = target_testing_data.astype(np.float32)

In [140]:
target_testing_data

,log2relT
1,0.126712
8,-0.048230
13,0.018737
15,0.245623
16,0.127100
...,...
1123,-0.058046
1125,-0.015160
1136,0.013140
1137,0.021239


In [141]:
fluxomic_training_data.drop(columns=["knockout_id"], inplace=True)
fluxomic_testing_data.drop(columns=["knockout_id"], inplace=True)
transcriptomics_training_data.drop(columns=["knockout_id", TARGET_NAME], inplace=True)
transcriptomics_testing_data.drop(columns=["knockout_id", TARGET_NAME], inplace=True)

# Scale the gene expression data
transcriptomics_scaler = preprocessing.StandardScaler().fit(
    transcriptomics_training_data
)
transcriptomics_training_data_scaled = transcriptomics_scaler.transform(
    transcriptomics_training_data
).astype(np.float32)

transcriptomics_testing_data_scaled = transcriptomics_scaler.transform(
    transcriptomics_testing_data
).astype(np.float32)

# Scale the fluxomic data
fluxome_scaler = sklearn.preprocessing.StandardScaler().fit(fluxomic_training_data)
fluxomic_training_data_scaled: np.ndarray = fluxome_scaler.transform(
    fluxomic_training_data
).astype(np.float32)
fluxomic_testing_data_scaled: np.ndarray = fluxome_scaler.transform(
    fluxomic_testing_data
).astype(np.float32)

proteomics_scaler = preprocessing.StandardScaler().fit(proteomics_training_data)

proteomics_training_data_scaled = proteomics_scaler.transform(
    proteomics_training_data
).astype(np.float32)
proteomics_testing_data_scaled = proteomics_scaler.transform(
    proteomics_testing_data
).astype(np.float32)

In [142]:
print("Fluxomic training data shape:", fluxomic_training_data_scaled.shape)
print(
    "Transcriptomics training data shape:", transcriptomics_training_data_scaled.shape
)
print("Proteomics training data shape:", proteomics_training_data_scaled.shape)
print("Target training data shape:", target_training_data.shape)

print("Fluxomic testing data shape:", fluxomic_testing_data_scaled.shape)
print("Transcriptomics testing data shape:", transcriptomics_testing_data_scaled.shape)
print("Proteomics testing data shape:", proteomics_testing_data_scaled.shape)
print("Target testing data shape:", target_testing_data.shape)

Fluxomic training data shape: (835, 1848)
Transcriptomics training data shape: (835, 6170)
Proteomics training data shape: (835, 1850)
Target training data shape: (835, 1)
Fluxomic testing data shape: (203, 1848)
Transcriptomics testing data shape: (203, 6170)
Proteomics testing data shape: (203, 1850)
Target testing data shape: (203, 1)


In [143]:
proteomics_model = init_single_view_model(
    input_dim=proteomics_training_data_scaled.shape[1], model_name="proteomics"
)

Summary of the single-view model proteomics


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 1850)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ proteomics_1 (Dense)            │ (None, 1000)           │     1,851,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ proteomics_2 (Dense)            │ (None, 1000)           │     1,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,853,001 (10.88 MB)

 Trainable params: 2,853,001 (10.88 MB)

 Non-trainable params: 0 (0.00 B)

In [144]:
def train_single_view_model(
    model: tf.keras.Model,
    training_data: np.ndarray,
    testing_data: np.ndarray,
    target_training_data: pd.Series,
    target_testing_data: pd.Series,
    weights_name: str,
    epochs: int = EPOCHS,
    batches: int = BATCHES,
    validation: float = VALIDATION,
    verbose: bool = True,
) -> tf.keras.Model:
    # ) -> dict[str, int]:
    model.fit(
        x=training_data,
        y=target_training_data,
        epochs=epochs,
        batch_size=batches,
        validation_split=validation,
        verbose=verbose,
    )
    model.save_weights(DATA_DIR + weights_name + ".weights.h5")
    mse, mae = model.evaluate(testing_data, target_testing_data, verbose=1)
    print(
        f"Score of the model proteomics - mean squared error: {mse}, mean absoulte error: {mae}"
    )
    # result =  {"mse": mse, "mae": mae}
    result = model
    return result

In [145]:
proteomics_model.compile(
    loss="mean_squared_error",
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=LEARNING_RATE,
        weight_decay=LEARNING_RATE / EPOCHS,
        momentum=MOMENTUM,
    ),
    metrics=["mean_absolute_error"],
)

In [146]:
proteomics_model = train_single_view_model(
    model=proteomics_model,
    training_data=proteomics_training_data_scaled,
    target_training_data=target_training_data,
    testing_data=proteomics_testing_data_scaled,
    target_testing_data=target_testing_data,
    weights_name="proteomics",
)

Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.4005 - mean_absolute_error: 0.5027 - val_loss: 0.0391 - val_mean_absolute_error: 0.1371
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.4226 - mean_absolute_error: 0.5173 - val_loss: 0.0674 - val_mean_absolute_error: 0.1945
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.4254 - mean_absolute_error: 0.5113 - val_loss: 0.0371 - val_mean_absolute_error: 0.1290
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3893 - mean_absolute_error: 0.5068 - val_loss: 0.2090 - val_mean_absolute_error: 0.4158
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.4658 - mean_absolute_error: 0.5462 - val_loss: 0.1547 - val_mean_absolute_error: 0.3459
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.4800 - mean_absolute_error: 0.5591 - val_loss: 0.1081 - val_mean_absolute_error: 0.2748
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.4658 - mean_absolute_error: 0.5473 - val_loss

In [147]:
proteomics_predictions = proteomics_model.predict(proteomics_testing_data_scaled)
proteomics_testing_growth = target_testing_data
proteomics_testing_growth = proteomics_testing_growth.to_numpy()
proteomics_testing_growth = proteomics_testing_growth.ravel()

proteomics_result = compute_metrics(proteomics_predictions, proteomics_testing_growth)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [148]:
proteomics_result

{'mse': 0.04198397696018219,
 'pearson': 0.11084765637817003,
 'spearman': 0.008641490054540082,
 'coverage': 1.0,
 'r_squared': -0.024250030517578125}

In [149]:
# Load the models
print("Loading model: fluxomics")
flux_model = load_single_view_model_from_file(
    training_data=fluxomic_training_data_scaled, fname="fluxomic"
)
print("Loading model: transcriptomics")
transcriptomics_model = load_single_view_model_from_file(
    training_data=transcriptomics_training_data, fname="gene_expression"
)

print("Loading model: proteomics")
proteomics_model = init_single_view_model(
    input_dim=proteomics_training_data_scaled.shape[1], model_name="proteomics"
)
proteomics_model.summary()
proteomics_model.load_weights(DATA_DIR + "proteomics.weights.h5")


Loading model: fluxomics
Summary of the single-view model fluxomic


Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 1848)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fluxomic_1 (Dense)              │ (None, 1000)           │     1,849,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fluxomic_2 (Dense)              │ (None, 1000)           │     1,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,851,001 (10.88 MB)

 Trainable params: 2,851,001 (10.88 MB)

...

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 1850)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ proteomics_1 (Dense)            │ (None, 1000)           │     1,851,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ proteomics_2 (Dense)            │ (None, 1000)           │     1,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,853,001 (10.88 MB)

 Trainable params: 2,853,001 (10.88 MB)

 Non-trainable params: 0 (0.00 B)

In [150]:

print("Loading model: multi-view model (GE+MF+P)")
# Initialize the multi-view model
triple_model = init_triple_view_model(
    input1_dim=fluxomic_training_data.shape[1],
    input2_dim=transcriptomics_training_data.shape[1],
    input3_dim=proteomics_training_data.shape[1],
    learning_rate=LEARNING_RATE,
    epochs=EPOCHS,
    momentum=MOMENTUM,
    neurons=15,
    model_1=flux_model,
    model_2=transcriptomics_model,
    model_3=proteomics_model,
)
triple_model.summary()
triple_model.compile(loss="mse", optimizer="adam", metrics=["mean_absolute_error"])

Loading model: multi-view model (GE+MF+P)


Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_17      │ (None, 1848)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_18      │ (None, 6170)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 1850)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_25       │ (None, 1)         │  2,851,001 │ input_layer_17[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_27       │ (None, 1)         │  7,173,001 │ input_layer_18[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_29       │ (None, 1)         │  2,853,001 │ input_layer_19[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 3)         │          0 │ functional_25[0]… │
│ (Concatenate)       │                   │            │ functional_27[0]… │
│                     │                   │            │ functional_29[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last_hidden (Dense) │ (None, 15)        │         60 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 1)         │         16 │ last_hidden[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,877,079 (49.12 MB)

 Trainable params: 12,877,079 (49.12 MB)

 Non-trainable params: 0 (0.00 B)

In [151]:
triple_model.fit(
    x=[
        fluxomic_training_data_scaled,
        transcriptomics_training_data_scaled,
        proteomics_training_data_scaled,
    ],
    y=target_training_data,
    epochs=EPOCHS,
    batch_size=BATCHES,
    validation_split=VALIDATION,
    verbose=True,
)
score = triple_model.evaluate(
    [
        fluxomic_testing_data_scaled,
        transcriptomics_testing_data_scaled,
        proteomics_testing_data_scaled,
    ],
    target_testing_data,
    verbose=1,
)
score

Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - loss: 0.5491 - mean_absolute_error: 0.6111 - val_loss: 0.0246 - val_mean_absolute_error: 0.1014
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0290 - mean_absolute_error: 0.1037 - val_loss: 0.0272 - val_mean_absolute_error: 0.1167
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0324 - mean_absolute_error: 0.1145 - val_loss: 0.0220 - val_mean_absolute_error: 0.1023
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0282 - mean_absolute_error: 0.1011 - val_loss: 0.0172 - val_mean_absolute_error: 0.0854
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0192 - mean_absolute_error: 0.0874 - val_loss: 0.0159 - val_mean_absolute_error: 0.0854
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0195 - mean_absolute_error: 0.0870 - val_loss: 0.0167 - val_mean_absolute_error: 0.0936
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0248 - mean_absolute_error: 0.0985 - val_los

[0.01998480223119259, 0.09011077135801315]

In [152]:
triple_model.save_weights(DATA_DIR + "multi_view_model_GE_MF_P.weights.h5")

In [153]:
print("Loading weights: multi_view_model_GE_MF_P.weights.h5")
triple_model.load_weights(DATA_DIR + "multi_view_model_GE_MF_P.weights.h5")

# Predict on the dataset
predictions = triple_model.predict(
    [
        fluxomic_testing_data_scaled,
        transcriptomics_testing_data_scaled,
        proteomics_testing_data_scaled,
    ]
)
results = compute_metrics(predictions, target_testing_data.to_numpy().ravel())

Loading weights: multi_view_model_GE_MF_P.weights.h5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [155]:
results

{'mse': 0.01998480223119259,
 'pearson': 0.7569092961790557,
 'spearman': 0.6608731577301458,
 'coverage': 1.0,
 'r_squared': 0.5124465227127075}